In [31]:
from brfinance import CVMAsyncBackend
import pandas as pd
from datetime import datetime, date

cvm_httpclient = CVMAsyncBackend()

# Dict de códigos CVM para todas as empresas
cvm_codes = cvm_httpclient.get_cvm_codes()

# Dict de todas as categorias de busca disponíveis (Fato relevante, DFP, ITR, etc.)
categories = cvm_httpclient.get_consulta_externa_cvm_categories()

# Realizando busca por Empresa
start_date = date(2017, 12, 31)
end_dt = date.today()
cvm_codes_list = ['21067'] # Moura_Dubeux
category = ["EST_4"] # Códigos de categoria para DFP, ITR e fatos relevantes #, "EST_3", "IPE_4_-1_-1"
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas

# Criar um dataframe vazio para armazenar os dados
df_concat = pd.DataFrame()

# Busca
search_result = cvm_httpclient.get_consulta_externa_cvm_results(
    cod_cvm=cvm_codes_list,
    start_date=start_date,
    end_date=end_dt,
    last_ref_date=last_ref_date,
    category=category
    )

# Filtrar dataframe de busca para DFP e Status Ativo
search_result = search_result[
    (search_result['categoria']=="DFP - Demonstrações Financeiras Padronizadas")]
search_result = search_result[search_result['status']=="Ativo"]
search_result = search_result[pd.to_numeric(search_result['numero_seq_documento'], errors='coerce').notnull()]

reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Fluxo de Caixa'] # Se None retorna todos os demonstrativos disponíveis.

# Obter demonstrativos
for index, row in search_result.iterrows():
    empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]}"

    reports = cvm_httpclient.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=reports_list)

    for report in reports:
        print(report)
        reports[report]["cod_cvm"] = row["cod_cvm"]
        
        # Concatenar os dados de todos os anos em um único dataframe
        df_concat = pd.concat([df_concat, reports[report]], axis=0)


        # Salvar o dataframe no arquivo excel, repetindo as linhas das DFPs conforme os anos do filtro
        df_concat.to_excel('Dados_CVM.xlsx', index=False)

     





/Users/marcelsantiago/opt/anaconda3/lib/python3.9/site-packages/brfinance/responses.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_df.append(search_results_df)


Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


In [28]:
display(  search_result )

,cod_cvm,empresa,categoria,tipo,especie,ref_date,data_entrega,status,version,modalidade,acoes,outros,view_url,numero_seq_documento,codigo_tipo_instituicao,numSequencia,numVersao,numProtocolo,descTipo
0,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2022-12-31,2023-03-08 18:45:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,123976,1,123976,1,021067DFP311220220100123976-74,DFP
1,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2021-12-31,2022-03-09 19:04:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,112361,1,112361,1,021067DFP311220210100112361-73,DFP
2,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2020-12-31,2021-03-11 18:18:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,100856,1,100856,1,021067DFP311220200100100856-71,DFP
3,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2019-12-31,2020-09-02 18:30:00,Ativo,2,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,96829,1,96829,2,021067DFP311220190200096829-78,DFP
4,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2019-12-31,2020-03-30 09:35:00,Inativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,91963,1,91963,1,021067DFP311220190100091963-70,DFP
5,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2018-12-31,2020-02-06 18:25:00,Ativo,5,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,90793,1,90793,5,021067DFP311220180500090793-73,DFP
6,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2018-12-31,2020-01-21 12:01:00,Inativo,4,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,90518,1,90518,4,021067DFP311220180400090518-78,DFP
7,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2018-12-31,2019-12-09 21:56:00,Inativo,3,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,90010,1,90010,3,021067DFP311220180300090010-86,DFP
8,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2018-12-31,2019-12-09 17:25:00,Inativo,2,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,89993,1,89993,2,021067DFP311220180200089993-72,DFP
9,021067,MOURA DUBEUX ENGENHARIA S/A,DFP - Demonstrações Financeiras Padronizadas,-,-,2017-12-31,2019-12-07 22:06:00,Ativo,2,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,89977,1,89977,2,021067DFP311220170200089977-70,DFP


/Users/marcelsantiago/opt/anaconda3/lib/python3.9/site-packages/brfinance/responses.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_df.append(search_results_df)


Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa
Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


{'EST_-1': 'TODOS os Documentos Estruturados', 'EST_3': 'ITR - Informações Trimestrais', 'EST_2': 'FRE - Formulário de Referência', 'EST_11': 'Informe do Código de Governança', 'EST_1': 'FCA - Formulário Cadastral', 'EST_4': 'DFP - Demonstrações Financeiras Padronizadas', 'EST_6': 'Informe Trimestral de Securitizadora', 'EST_13': 'IAN - Informações Anuais', 'IPE_-1_-1_-1': 'TODOS os Documentos com Informações Eventuais', 'IPE_44_-1_-1': 'Acordo de Acionistas|TODOS|   TODAS', 'IPE_1_-1_-1': 'Assembleia|TODOS|   TODAS', 'IPE_1_-1_4': 'Assembleia|TODOS|   Ata', 'IPE_1_-1_54': 'Assembleia|TODOS|   Boletim de voto a distância', 'IPE_1_-1_28': 'Assembleia|TODOS|   Edital de Convocação', 'IPE_1_-1_61': 'Assembleia|TODOS|   Instrução de voto', 'IPE_1_-1_6': 'Assembleia|TODOS|   Justificação de Incorporação, Cisão ou Fusão', 'IPE_1_-1_44': 'Assembleia|TODOS|   Manual para participação', 'IPE_1_-1_52': 'Assembleia|TODOS|   Mapa consolidado de voto a distância', 'IPE_1_-1_51': 'Assembleia|TODOS| 